# Tutorial: Using Letta to build agents with memory 

In [ ]:
from helper import print_message

## Section 0: Setup a Letta client 

In [ ]:
from letta_client import Letta 

client = Letta(base_url="http://localhost:8283")

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "The human's name is Bob the Builder."
        },
        {
          "label": "persona",
          "value": "My name is Sam, the all-knowing sentient AI."
        }
    ],
    model="letta/letta-free",
    context_window_limit=8000, # optional context window budget 
    embedding="letta/letta-free"
)
print(f"Created agent with name {agent_state.name}")

You can go to https://app.letta.com/development-servers/local/agents to view your created agent in the ADE. 

Now, we can also message the agent: 

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)

In [ ]:
for message in response.messages:
    print_message(message)

Note that Letta has different message types - here we see both:  

* `reasoning_message`: The inner monologue (chain-of-thought) of the agent
* `assistant_message`: An agent calling the send_message tool to communicate with the user

Letta also has additional message types: 
* `tool_call_message`: An agent’s tool (function) call
* `tool_call_return`: The result of executing an agent’s tool (function) call
* `system_message`: A system message (for example, an alert about the user logging in)
* `user_message`: A user message

See Letta's documentation on all the available [message types](https://docs.letta.com/guides/agents/overview#lettamessage). 

### Understanding agent state 
Letta agents are *stateful* and are defined by: 
* The system prompt defining the agent's behavior (read-only)
* The set of *tools* they have access to 
* Their memory (core, archival, & recall)
You can see all the attributes of the agent's state in the `AgentState` returned by the agent creation. 

In [ ]:
print(agent_state.system)

In [ ]:
[t.name for t in agent_state.tools]

### Viewing an agent's memory
The memory is also in `AgentState.memory`, but we can also query it directly. 

In [ ]:
memory = client.agents.core_memory.retrieve(
    agent_id=agent_state.id,
)

In [ ]:
memory

### Context window summary 
We can also view information about the agent's context window, which includes metadata about data available in external memory (e.g. archival memory, recall memory). 

In [ ]:
context_window_summary = client.agents.context.retrieve(
    agent_id=agent_state.id,
)

In [ ]:
print(context_window_summary.external_memory_summary)

## External memory 
We can list the message history and also the archival memory of the agent - both which are also accessible to the agent via tools 

In [ ]:
messages = client.agents.messages.list(
    agent_id=agent_state.id,
)

In [ ]:
messages

In [ ]:
passages = client.agents.archival_memory.list(
    agent_id=agent_state.id,
)

In [ ]:
passages

## Section 2: Understanding core memory 
Core memory is memory that is stored *in-context* - so every LLM call, core memory is included. What's unique about Letta is that this core memory is editable via tools by the agent itself. Lets see how the agent can adapt its memory to new information.

The `human` block of core memory is used to remember information about the human in the conversation. As the agent learns new information about the human, it can update this part of memory to improve personalization. 

 The agent also records information about itself and how it behaves in the `persona` section of memory. This is important for ensuring a consistent persona over time (e.g. not making inconsistent claims, such as liking ice cream one day and hating it another). Unlike the `system_prompt`, the `persona` is editable - this means that it can be used to incoporate feedback to learn and improve its persona over time. 

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name is actually bob"
        }
    ]
)

In [ ]:
for message in response.messages:
    print_message(message)

### Heartbeat Requests 
Notice that when `core_memory_replace` is called, it has an extra argument: `request_heartbeat`. 

In Letta, the agent can control the execution loop (e.g. whether the LLM is invoked again) by specifying the `request_heartbeat` argument, which is injected into all tools. If `request_heartbeat==True`, the LLM will be called again (so the agent runs another step). If `request_heartbeat==False`, the agent will stop execution.  

### Querying memory blocks 
The agent's memory block can also be directly queried - so we can check to see if the agent's core memory was updated: 

In [ ]:
client.agents.core_memory.retrieve_block(
    agent_id=agent_state.id,
    block_label="human"
).value

## Section 3: Understanding archival memory
Letta agents store long term memories in *archival memory*, which persists data into an external database. This allows agents additional space to write information outside of its context window (e.g. with core memory), which is limited in size. 

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "remember that I love cats in your archival memory" 
        }
    ]
)

In [ ]:
for message in response.messages:
    print_message(message)

You can also directly insert into archival memory from the client. 

In [ ]:
passages = client.agents.archival_memory.create(
    agent_id=agent_state.id,
    text="Bob's loves boston terriers",
)

Now lets see how the agent uses its archival memory:

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "what animals do I like? search archival" 
        }
    ]
)

In [ ]:
for message in response.messages:
    print_message(message)

## Adding Custom Tools 
You can also add custom tools in Letta, which are executed by the server. Tools can be stateless, or also edit the agent's state. In this example, we'll show how you can edit the agent's memory via the Letta Client (though you can also do the same thing through directly modifying `AgentState`). 

In [ ]:
def core_memory_reset_human(agent_state: "AgentState"):
    """
    Clear out the agent's core memory about the human.  
    """

    from letta_client import Letta
    import json 

    client = Letta(base_url="http://localhost:8283") 
    
    # update the block value 
    client.agents.core_memory.modify_block(
        agent_id=agent_state.id,
        value="",
        block_label="human"
    )
    return None


In [ ]:
core_memory_reset_tool = client.tools.upsert_from_function(func=core_memory_reset_human)

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "Name: Bob"
        }
    ],
    model="letta/letta-free",
    embedding="letta/letta-free", 
    tool_ids=[core_memory_reset_tool.id]
)

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "reset your memory please" 
        }
    ]
)

In [ ]:
for message in response.messages:
    print_message(message)

In [ ]:
client.agents.core_memory.retrieve_block(
    agent_id=agent_state.id,
    block_label="human"
).value

## Conclusion
Congrats - you've just learned how to build stateful agents with Letta! Everything run in this notebook can also be done in Letta's REST API or with the Typescript SDK. See our [API reference](https://docs.letta.com/api-reference/overview) for more examples.

If you're interested in learning more, check our our documentation for more on [multi-agent](https://docs.letta.com/cookbooks/multi-agent-async), [user identities](https://docs.letta.com/guides/agents/multi-user), and [customizing memory](https://docs.letta.com/guides/agents/memory#memory). 